In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
tf.compat.v1.disable_eager_execution() #  Eager Execution を無効にする
# ART 関連
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import KerasClassifier

##  モデルの作成

In [2]:
robust_model = Sequential()
robust_model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(30,30,3)))
robust_model.add(MaxPool2D(pool_size=(2, 2)))
robust_model.add(Dropout(rate=0.25))
robust_model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
robust_model.add(MaxPool2D(pool_size=(2, 2)))
robust_model.add(Dropout(rate=0.25))
robust_model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
robust_model.add(Flatten())
robust_model.add(Dense(units=64, activation='relu'))
robust_model.add(Dense(43, activation='softmax'))
# Compiling the model
robust_model.compile(
     loss='categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### モデルをロードする場合は以下のプログラムを実行

In [3]:
#モデルを読み込む場合
#robust_model = tf.keras.models.load_model("モデル名を指定")

### テストデータの取得

In [4]:
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
x_test = x_test/255
y_test = np.argmax(y_test,axis=1)
classifier= KerasClassifier(model=robust_model, clip_values=(0, 1))

### 対象クラスの抽出

In [5]:
target_class = []
error_ratios = []
attack = ProjectedGradientDescent(classifier,eps=(8/255), eps_step=(2/255), max_iter=10, batch_size=64)
adv_check_images =  attack.generate(x_test)      
y_pred = robust_model.predict(adv_check_images)
y_pred_classes = np.argmax(y_pred, axis=1)
for i in range(43):
    class_indices = (y_test == i)
    total_count = class_indices.sum()
    error_count = (y_test[class_indices] != y_pred_classes[class_indices]).sum()
    error_ratio = error_count / total_count
    error_ratios.append((i,1-error_ratio))
sorted_errors = sorted(error_ratios, key=lambda x: x[1], reverse=False)  
for i in range(43):
    if sorted_errors[i][1] <= 0.2:
        target_class.append(sorted_errors[i][0])

In [6]:
print(target_class)

[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 40, 41, 42, 39, 7]
